In [1]:
pip install python-telegram-bot tensorflow pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.7/662.7 kB 16.9 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
from PIL import Image
import io
import nest_asyncio
from telegram import Bot

In [ ]:
nest_asyncio.apply() #асинхронная работа бота

model = torch.load('/content/drive/MyDrive/ИИ/V3/efficientnet_b0_full.pth') # загрузка ии

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # загрузка ии
model.to(device)# загрузка ии

model.eval()# загрузка ии

unique_labels = ["Злость", "Грусть", "Счастье"] #классы которые может определить ИИ

def process_image(image):#работа с изображением
    transform = transforms.Compose([ #работа с изображением
        transforms.Resize((224, 224)),#работа с изображением
        transforms.ToTensor(),#работа с изображением
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])#работа с изображением
    ])
    image_tensor = transform(image).unsqueeze(0)#работа с изображением
    return image_tensor#работа с изображением

API_TOKEN = "8100399006:AAH5NnFdFmyjdlDhUnG8gqICewcUOpLaclQ" #токен бота
bot = Bot(token=API_TOKEN) #пробуждение бота

async def start(update: Update, context: CallbackContext) -> None: #команды для тг
    await update.message.reply_text("Привет! Я бот для классификации эмоций на изображениях. Отправь мне изображение, и я попробую его классифицировать!")

async def handle_image(update: Update, context: CallbackContext) -> None: #ожидание изображения
    photo = update.message.photo[-1]
    file_info = await bot.get_file(photo.file_id)
    downloaded_file = await file_info.download_as_bytearray()
    image = Image.open(io.BytesIO(downloaded_file))
    processed_image = process_image(image)
    processed_image = processed_image.to(device)

    with torch.no_grad():
        predictions = model(processed_image)
        probabilities = F.softmax(predictions, dim=1)
        predicted_index = torch.argmax(probabilities, dim=1).item()
        predicted_label = unique_labels[predicted_index]
        confidence = probabilities[0][predicted_index].item()

    await update.message.reply_text(f"Класс: {predicted_label}")

def main(): # определение команд бота
    application = Application.builder().token(API_TOKEN).build()
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_image))
    application.run_polling()

if __name__ == "__main__": #запуск бота
    main()

<ipython-input-3-153c2c265cdd>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/ИИ/V3/efficientnet_b0_full.pth') # загрузка ии
